In [97]:
from collections import defaultdict
from copy import deepcopy
from pprint import pprint
from tqdm import tqdm

In [98]:
q = 7
F.<a> = GF(q)
d = 3

R.<x> = PolynomialRing(F)
p = F.characteristic()
r = F.degree()

# PGL = [(x + b)/(c*x + d) for b in F for c in F for d in F if d - b*c != 0] + [1 / (c*x + d) for c in F for d in F if c != 0]

def check(s, t):
    poly = x^4 - 2*x^3 - s*x^2 - 2*t*x + t
    return not any(root[0]^3 != -t for root in poly.roots())

# deg3ratls = set(f / (x^3 + g) for f in R.polynomials(max_degree=2) for g in R.polynomials(max_degree=2) if gcd(f, x^3 + g) == 1)
deg3ratls = set((x^3 + s*x + t) / (x * (x - 1)) for s in F for t in F if t != 0 and 1 + s + t != 0 and check(s, t))

In [94]:
def numRatlEquivClasses(p, q, n): 
    def A(q, n): 
        a = sum([euler_phi(d) * (q^(2*n/d-2) + (d-1)*(q^(2*n/d)-1)/(q+1)) for d in divisors(gcd(q-1, n)) if d > 1])
        b = sum([euler_phi(d) * (q^(2*floor(n/d)+1) + 1)/(q+1) for d in divisors(q-1) if (d not in divisors(n))])
        return a + b 

    def B(q, n):
        a = sum([euler_phi(d) * (q^(2*n/d-2) + (q+1) * (q^(2*n/d) - (-1)^(n/d))/(q^2 + 1)) for d in divisors(gcd((q+1), n)) if d % 2 == 0])
        b = sum([euler_phi(d) * q^(2*n/d-2) for d in divisors(gcd((q+1), n)) if (d % 2 == 1 and d > 1)])
        c = 1/((q+1)*(q^2+1)) * sum([euler_phi(d) * ((1+(-1)^(n//d))/2 * (1+q)^2 + q*(q^(2 * n//d + 2) -1)) for d in divisors(q+1) if d not in divisors(n)])
        d = 1/(q^2 + 1) * sum([euler_phi(d) * ((-1)^(n//d) * (1+q) + q^(2 * (n//d) + 1) * (q-1)) for d in divisors(q+1) if d in divisors(q+1) if d not in divisors(n)])
        # print("B terms are", a, b, c, d)
        return a + b + d

    def C(q, n):
        if n % p == 0:
            return q^(2*n/p-1) 
        elif n == 1: 
            return 1 
        elif n % p == 1: 
            return q^(2*(n-1)/p-1) * (q-1) 
        else: 
            return 0

    # print("Each components is", A(q, n), B(q, n), C(q, n))
    
    return (q^(2*n-3))/(q^2-1) + 1/(2*(q-1)) * A(q, n) + 1/(2*(q+1)) * B(q, n) + 1/q * C(q,n)

In [95]:
print(numRatlEquivClasses(p, q, d))

16


In [ ]:
# very slow code; pls scratch this later on but I needed some data for very small case 
def ratlEquiv(f, g):
    """returns whether or not f is PGL-equiv to g""" 
    for alpha in PGL: 
        for beta in PGL: 
            try: 
                if alpha(f(beta)) == g: 
                    return (true, alpha, beta)
            except ZeroDivisionError: 
                continue
    return False

def valueFreq(f): 
    """given a rational function f, compute the value frequency of it"""
    frequencies = defaultdict(int)
    for a in F:
        try: 
            value = f(a)
        except ZeroDivisionError: 
            value = "infinity" 
        frequencies[value] += 1
    return sorted(frequencies.values()) 

In [99]:
def reallyslowRatlEquivClasses(print_progress=False): 
    classes = defaultdict(set)
    total = len(deg3ratls)
    candidates = deepcopy(deg3ratls)
    if print_progress:
        print(f"{total - len(candidates)} out of {total} done", end="\r")
    while candidates:
        f = candidates.pop()
        classes[f].add(f)
        for alpha in PGL: 
            for beta in PGL: 
                try: 
                    g = alpha(f(beta))
                    if g in candidates:
                        classes[f].add(g)
                        candidates.discard(g)
                except ZeroDivisionError: 
                    continue
        # classes.add(f)
        if print_progress:
            print(f"{total - len(candidates)} out of {total} done", end="\r")
    if print_progress:
        print()
    return classes

In [100]:
classes = reallyslowRatlEquivClasses(print_progress=True)
print(len(classes))
pprint(classes)

# value_freqs = []
# for f in tqdm(classes):
#     value_freqs.append((f, valueFreq(f)))
# pprint(value_freqs)

TypeError: no common canonical parent for objects with parents: 'Finite Field of size 7' and 'Fraction Field of Univariate Polynomial Ring in x over Finite Field of size 17'

In [101]:
def getOrbit(f):
    s = f.numerator()[1]
    t = f.numerator()[0]

    preimages = defaultdict(set)
    for a in F:
        if a != 0 and a != 1:
            preimages[f(a)].add(a)
    
    pairs = { (s, t) }
    for preimage in preimages.values():
        if len(preimage) == 3:
            u, v, w = preimage
            s_prime = (u^2*(-1 + v)*v*(u - w) - u*(-1 + v)*v^2*(u - w) + u*(u - v)*v^2*(-1 + w) + (1 - v)*v^2*(u - w)*w + (-1 + v)*v^2*(u - w)*w - u^2*(u - v)*(-1 + w)*w + v^2*(-u + v)*(-1 + w)*w - u*(-1 + v)*(u - w)*w^2 + (-1 + v)*v*(u - w)*w^2 + u*(u - v)*(-1 + w)*w^2 + (u - v)*v*(-1 + w)*w^2 + v*(-u + v)*(-1 + w)*w^2) / (u^2*(u - v)*(-1 + w)*w - 2*u*(u - v)*v*(-1 + w)*w + (u - v)*v^2*(-1 + w)*w)
            t_prime = (-(u^2*(-1 + v)*v*(u - w)) + 2*u*(-1 + v)*v*(u - w)*w - (-1 + v)*v*(u - w)*w^2)/(u^2*(u - v)*(-1 + w)*w - 2*u*(u - v)*v*(-1 + w)*w + (u - v)*v^2*(-1 + w)*w)
            pairs.add((s_prime, t_prime))

    # print(pairs)
    
    def get(s, t):
        return {
            (s, t),
            (s / t, 1 / t),
            (s, -1 - s - t),
            (s / t, -(1 + s + t) / t),
            (-s / (1 + s + t), -1 / (1 + s + t)),
            (-s / (1 + s + t), -t / (1 + s + t))
        }
    
    pairs = set.union(*(get(s, t) for s, t in pairs))
    return set((x^3 + s*x + t) / (x^2 - x) for s, t in pairs)

def ratlEquivClassesCase3(print_progress=False):
    classes = defaultdict(set)
    total = len(deg3ratls)
    candidates = deepcopy(deg3ratls)
    if print_progress:
        print(f"{total - len(candidates)} out of {total} done", end="\r")
    while candidates:
        f = candidates.pop()
        orbit = getOrbit(f)
        classes[f] = orbit
        candidates -= orbit
        if print_progress:
            print(f"{total - len(candidates)} out of {total} done", end="\r")
    if print_progress:
        print()
    return classes

In [68]:
getOrbit((x^3 + 3*x + 6) / (x^2 - x))

{(3, 4), (3, 6)}


{(x^3 + 6*x + 2)/(x^2 + 6*x),
 (x^3 + 4*x + 3)/(x^2 + 6*x),
 (x^3 + 3*x + 4)/(x^2 + 6*x),
 (x^3 + 6*x + 5)/(x^2 + 6*x),
 (x^3 + 3*x + 6)/(x^2 + 6*x),
 (x^3 + 4*x + 6)/(x^2 + 6*x)}

In [102]:
classes = ratlEquivClassesCase3(print_progress=True)
print(len(classes))
pprint(classes)

19 out of 19 done
6
defaultdict(<class 'set'>,
            {(x^3 + 1)/(x^2 + 6*x): {(x^3 + 1)/(x^2 + 6*x),
                                     (x^3 + 3)/(x^2 + 6*x),
                                     (x^3 + 5)/(x^2 + 6*x)},
             (x^3 + 3*x + 1)/(x^2 + 6*x): {(x^3 + 3*x + 1)/(x^2 + 6*x),
                                           (x^3 + 3*x + 2)/(x^2 + 6*x),
                                           (x^3 + 5*x + 4)/(x^2 + 6*x)},
             (x^3 + 4*x + 1)/(x^2 + 6*x): {(x^3 + 4*x + 1)/(x^2 + 6*x)},
             (x^3 + 6*x + 2)/(x^2 + 6*x): {(x^3 + 6*x + 2)/(x^2 + 6*x),
                                           (x^3 + 4*x + 3)/(x^2 + 6*x),
                                           (x^3 + 3*x + 4)/(x^2 + 6*x),
                                           (x^3 + 6*x + 5)/(x^2 + 6*x),
                                           (x^3 + 3*x + 6)/(x^2 + 6*x),
                                           (x^3 + 4*x + 6)/(x^2 + 6*x)},
             (x^3 + 3*x + 5)/(x^2 + 6*x): {(x^3 +

In [58]:
s, t = F(2), F(3)
ratlEquiv((x^3 + (-s / (1 + s + t))*x + (-t / (1 + s + t)))/(x^2 - x), (x^3 + s*x + t)/(x^2 - x))

(True, x, x)

In [15]:
pprint([(x^3 - a).roots() for a in F])

[[(0, 3)], [(4, 1), (2, 1), (1, 1)], [], [], [], [], [(6, 1), (5, 1), (3, 1)]]
